In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print("Device:", device)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe_translate = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device=device)

test_text = "Bonjour, je suis Quentin, comment allez-vous aujourd'hui ? J'ai un problème médical dont j'aimerai discuter."
translation = pipe_translate(test_text)
print("Translation:", translation)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe = pipeline("text-to-speech", model="suno/bark-small", device=device)
# pipe = pipeline("text-to-speech", model="suno/bark", device=device)
pipe_tts = pipeline("text-to-speech", model="facebook/mms-tts-eng", device=device) # Better quality, way faster than bark

speech = pipe_tts(translation[0]["translation_text"])

import IPython
IPython.display.Audio(speech["audio"], rate=speech["sampling_rate"])

In [20]:
import gradio as gr
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor
import torch
import numpy as np

device = "cuda:0" if torch.cuda.is_available() else "cpu"

device = "cpu"
torch_dtype = torch.float16 if device != "cpu" else torch.float32

print("Device:", device)

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe_transcription = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)
pipe_translate = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device=device)
pipe_tts = pipeline("text-to-speech", model="facebook/mms-tts-eng", device=device) # Better quality, way faster than bark


def get_translation(text):
    return pipe_translate(text)[0]["translation_text"]

def get_transcript(voice):
    return pipe_transcription(voice, generate_kwargs={"task": "translate", "language": "french"})["text"]

def get_audio(text):
    speech = pipe_tts(text)
    return speech["sampling_rate"], (speech["audio"]* 32767).astype(np.int16).T

with gr.Blocks() as demo:    
    with gr.Tab("Voix (plus lent)"):
        voice = gr.Audio(sources=["microphone"], type="filepath")

        translation_button = gr.Button("Traduire votre enregistrement !")
        output_text = gr.Textbox(
                    label="Texte traduit",
                    info="Votre texte",
                    lines=3,
                    placeholder="Votre traduction",
                )

        speech_button = gr.Button("Générer audio !")
        
        translation_button.click(
                get_transcript,
                inputs=[
                    voice
                ],
                outputs=[
                    output_text
                ],
        )
        speech_button.click(
                get_audio,
                inputs=[
                    output_text
                ],
                outputs=[
                    gr.Audio(label="Output")
                ],
        )
    with gr.Tab("Texte (rapide)"):
        input_text = gr.Textbox(
                    label="Input text",
                    info="Your text",
                    lines=3,
                    placeholder="Écrire le texte à traduire",
                )
        translation_button = gr.Button("Traduire...")
        output_text = gr.Textbox(
                    label="Output text",
                    info="Your text",
                    lines=3,
                    placeholder="Votre traduction",
                )
        speech_button = gr.Button("Générer audio...")
        translation_button.click(
                get_translation,
                inputs=[
                    input_text
                ],
                outputs=[
                    output_text
                ],
        )
        speech_button.click(
                get_audio,
                inputs=[
                    output_text
                ],
                outputs=[
                    gr.Audio(label="Output")
                ],
        )

demo.launch()

Device: cpu


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/qbateux/pytorch/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', '

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
